<a href="https://colab.research.google.com/github/shaheen101/myPackage/blob/master/ZINDI_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn.metrics import mean_squared_error
import sklearn.metrics as metrics
from sklearn.svm import SVR
import lightgbm as lgb

from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgbm

sns.set(style='white', context='notebook', palette='deep')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [17]:
from google.colab import files
import io
Train_df= pd.read_csv(io.BytesIO(uploaded['Train.csv']))
Train_df.head()



,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [20]:
Test_df = pd.read_csv(io.BytesIO(uploaded['Test.csv']))
Test_df.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,4:44:29 PM,27,3,4:53:04 PM,27,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,12:59:17 PM,17,5,1:20:27 PM,17,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14 AM,27,4,11:25:05 AM,27,4,11:33:20 AM,27,4,11:57:54 AM,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,1:51:35 PM,17,1,1:53:27 PM,17,1,2:02:41 PM,17,1,2:16:52 PM,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28 AM,11,2,11:34:45 AM,11,2,11:47:19 AM,11,2,11:56:04 AM,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858


In [22]:
Rider_df = pd.read_csv(io.BytesIO(uploaded['Riders.csv']))
Rider_df.head()


,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159
1,Rider_Id_479,360,951,13.5,176
2,Rider_Id_648,1746,821,14.3,466
3,Rider_Id_753,314,980,12.5,75
4,Rider_Id_335,536,1113,13.7,156


In [23]:
print(Train_df.shape, Test_df.shape, Rider_df.shape)
Train_df.head()

(21201, 29) (7068, 25) (960, 5)


,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [24]:
for col in Train_df.columns:
    if col not in Test_df.columns:
        print(col)

Arrival at Destination - Day of Month
Arrival at Destination - Weekday (Mo = 1)
Arrival at Destination - Time
Time from Pickup to Arrival


In [25]:
#Checkingtarget variable was calculated

labels = pd.to_datetime(Train_df['Arrival at Destination - Time']) - pd.to_datetime(Train_df['Pickup - Time'])
list(labels.dt.seconds[:5]), list(Train_df['Time from Pickup to Arrival'][:5])

([745, 1993, 455, 1341, 1214], [745, 1993, 455, 1341, 1214])

In [26]:
#Drop data not available in test, Pickup Time + label = Arrival times


Train_df = Train_df.drop(['Arrival at Destination - Day of Month', 'Arrival at Destination - Weekday (Mo = 1)', 'Arrival at Destination - Time'], axis=1)

In [29]:
#Create (full_df = train + test) 
border = Train_df.shape[0]
Test_df['Time from Pickup to Arrival'] = [np.nan]* Test_df.shape[0]
combined_df = pd.concat([Train_df, Test_df], axis=0, ignore_index=True)

Train_df.shape, Test_df.shape, combined_df.shape

((21201, 26), (7068, 26), (28269, 26))

In [30]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28269 entries, 0 to 28268
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Order No                              28269 non-null  object 
 1   User Id                               28269 non-null  object 
 2   Vehicle Type                          28269 non-null  object 
 3   Platform Type                         28269 non-null  int64  
 4   Personal or Business                  28269 non-null  object 
 5   Placement - Day of Month              28269 non-null  int64  
 6   Placement - Weekday (Mo = 1)          28269 non-null  int64  
 7   Placement - Time                      28269 non-null  object 
 8   Confirmation - Day of Month           28269 non-null  int64  
 9   Confirmation - Weekday (Mo = 1)       28269 non-null  int64  
 10  Confirmation - Time                   28269 non-null  object 
 11  Arrival at Pick

In [31]:
#Renaming columns (shorten, remove space, standardize)
new_names = {"Order No": "Order_No", "User Id": "User_Id", "Vehicle Type": "Vehicle_Type",
    "Personal or Business": "Personal_Business", "Placement - Day of Month": "Pla_Mon",
    "Placement - Weekday (Mo = 1)": "Pla_Weekday", "Placement - Time": "Pla_Time", 
    "Confirmation - Day of Month":"Con_Day_Mon", "Confirmation - Weekday (Mo = 1)": "Con_Weekday","Confirmation - Time": "Con_Time", 
    "Arrival at Pickup - Day of Month": "Arr_Pic_Mon", "Arrival at Pickup - Weekday (Mo = 1)": "Arr_Pic_Weekday", 
                "Arrival at Pickup - Time": "Arr_Pic_Time", "Platform Type": "Platform_Type",
     "Pickup - Day of Month": "Pickup_Mon", "Pickup - Weekday (Mo = 1)": "Pickup_Weekday",           
    "Pickup - Time": "Pickup_Time",  "Distance (KM)": "Distance(km)",
    "Precipitation in millimeters": "Precipitation(mm)", "Pickup Lat": "Pickup_Lat", "Pickup Long": "Pickup_Lon", 
    "Destination Lat": "Destination_Lat", "Destination Long":"Destination_Lon", "Rider Id": "Rider_Id",
                            "Time from Pickup to Arrival": "Time_Pic_Arr"
                           }
combined_df = combined_df.rename(columns=new_names)
combined_df.columns

Index(['Order_No', 'User_Id', 'Vehicle_Type', 'Platform_Type',
       'Personal_Business', 'Pla_Mon', 'Pla_Weekday', 'Pla_Time',
       'Con_Day_Mon', 'Con_Weekday', 'Con_Time', 'Arr_Pic_Mon',
       'Arr_Pic_Weekday', 'Arr_Pic_Time', 'Pickup_Mon', 'Pickup_Weekday',
       'Pickup_Time', 'Distance(km)', 'Temperature', 'Precipitation(mm)',
       'Pickup_Lat', 'Pickup_Lon', 'Destination_Lat', 'Destination_Lon',
       'Rider_Id', 'Time_Pic_Arr'],
      dtype='object')

In [32]:
#Convert Time from 12H to 24H
def convert_to_24hrs(fulldf):
    for col in fulldf.columns:
        if col.endswith("Time"):
            fulldf[col] = pd.to_datetime(fulldf[col], format='%H:%M:%S %p').dt.strftime("%H:%M:%S")
    return fulldf

combined_df = convert_to_24hrs(combined_df)

combined_df[['Pla_Time', 'Con_Time' , 'Arr_Pic_Time', 'Pickup_Time']][3:6]

,Pla_Time,Con_Time,Arr_Pic_Time,Pickup_Time
3,09:25:34,09:26:05,09:37:56,09:43:06
4,09:55:18,09:56:18,10:03:53,10:05:23
5,03:07:35,03:08:57,03:21:36,03:30:30


In [33]:
month_cols = [col for col in combined_df.columns if col.endswith("Mon")]
weekday_cols = [col for col in combined_df.columns if col.endswith("Weekday")]

count = 0
instances_of_different_days = [];
for i, row in combined_df.iterrows():
    if len(set(row[month_cols].values)) > 1:
        print(count+1, end="\r")
        count = count + 1
        instances_of_different_days.append(list(row[month_cols].values))
instances_of_different_days

[[17, 18, 18, 18], [11, 13, 13, 13]]

In [24]:
#Day & Weekday values are repeated in all rows except 2, we retain only one
#All Vehicle types are Bikes, Vehicle Type is not necessary.

combined_df['Day_of_Month'] = combined_df[month_cols[0]]
combined_df['Day_of_Week'] = combined_df[weekday_cols[0]]

combined_df.drop(month_cols+weekday_cols, axis=1, inplace=True)
combined_df.drop('Vehicle_Type', axis=1, inplace=True)

combined_df.head(3)

,Order_No,User_Id,Platform_Type,Personal_Business,Pla_Time,Con_Time,Arr_Pic_Time,Pickup_Time,Distance(km),Temperature,Precipitation(mm),Pickup_Lat,Pickup_Lon,Destination_Lat,Destination_Lon,Rider_Id,Time_Pic_Arr,Day_of_Month,Day_of_Week
0,Order_No_4211,User_Id_633,3,Business,09:35:46,09:40:10,10:04:47,10:27:30,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745.0,9,5
1,Order_No_25375,User_Id_2285,3,Personal,11:16:16,11:23:21,11:40:22,11:44:09,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993.0,12,5
2,Order_No_1899,User_Id_265,3,Business,12:39:25,12:42:44,12:49:34,12:53:03,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455.0,30,2


In [34]:
combined_df['Temperature'] = combined_df['Temperature'].fillna(combined_df['Temperature'].mean())
combined_df['Precipitation(mm)'].fillna(combined_df['Precipitation(mm)'].mean(), inplace=True)

In [35]:
numeric_cols = []
object_cols = []
time_cols = []
for k, v in combined_df.dtypes.items():
    if (v != object):
        if (k != "Time_Pic_Arr"):
            numeric_cols.append(k)
    elif k.endswith("Time"):
        time_cols.append(k)
    else:
        object_cols.append(k)
combined_df[numeric_cols].head(3) 

,Platform_Type,Pla_Mon,Pla_Weekday,Con_Day_Mon,Con_Weekday,Arr_Pic_Mon,Arr_Pic_Weekday,Pickup_Mon,Pickup_Weekday,Distance(km),Temperature,Precipitation(mm),Pickup_Lat,Pickup_Lon,Destination_Lat,Destination_Lon
0,3,9,5,9,5,9,5,9,5,4,20.400000,7.573502,-1.317755,36.830370,-1.300406,36.829741
1,3,12,5,12,5,12,5,12,5,16,26.400000,7.573502,-1.351453,36.899315,-1.295004,36.814358
2,3,30,2,30,2,30,2,30,2,3,23.255689,7.573502,-1.308284,36.843419,-1.300921,36.828195


In [36]:
combined_df[time_cols].head(3)

,Pla_Time,Con_Time,Arr_Pic_Time,Pickup_Time
0,09:35:46,09:40:10,10:04:47,10:27:30
1,11:16:16,11:23:21,11:40:22,11:44:09
2,12:39:25,12:42:44,12:49:34,12:53:03


In [37]:
#Convert an object to numeric (encoding), try (OHE -1,0)

le = LabelEncoder()
le.fit(combined_df['Personal_Business'])
combined_df['Personal_Business'] = le.transform(combined_df['Personal_Business'])
combined_df['Personal_Business'][:2]

0    0
1    1
Name: Personal_Business, dtype: int64

In [38]:
#Correlation between numeric features and the target variable
corr_matrix = combined_df[:border][numeric_cols+['Time_Pic_Arr']].corr()
print(corr_matrix["Time_Pic_Arr"].sort_values(ascending=False))

Time_Pic_Arr         1.000000
Distance(km)         0.580608
Destination_Lon      0.070425
Pickup_Lon           0.060285
Con_Weekday          0.009744
Arr_Pic_Weekday      0.009744
Pickup_Weekday       0.009744
Pla_Weekday          0.009693
Temperature          0.005779
Precipitation(mm)    0.003934
Platform_Type       -0.003827
Pickup_Mon          -0.014701
Arr_Pic_Mon         -0.014701
Con_Day_Mon         -0.014701
Pla_Mon             -0.014710
Pickup_Lat          -0.053823
Destination_Lat     -0.061872
Name: Time_Pic_Arr, dtype: float64


In [39]:
features = numeric_cols + ['Personal_Business']

data_df = combined_df[features]

y = combined_df[:border]['Time_Pic_Arr']
train = data_df[:border]
test = data_df[border:]

test.head()

,Platform_Type,Pla_Mon,Pla_Weekday,Con_Day_Mon,Con_Weekday,Arr_Pic_Mon,Arr_Pic_Weekday,Pickup_Mon,Pickup_Weekday,Distance(km),Temperature,Precipitation(mm),Pickup_Lat,Pickup_Lon,Destination_Lat,Destination_Lon,Personal_Business
21201,3,27,3,27,3,27,3,27,3,8,23.255689,7.573502,-1.333275,36.870815,-1.305249,36.822390,0
21202,3,17,5,17,5,17,5,17,5,5,23.255689,7.573502,-1.272639,36.794723,-1.277007,36.823907,0
21203,3,27,4,27,4,27,4,27,4,5,22.800000,7.573502,-1.290894,36.822971,-1.276574,36.851365,0
21204,3,17,1,17,1,17,1,17,1,5,24.500000,7.573502,-1.290503,36.809646,-1.303382,36.790658,0
21205,3,11,2,11,2,11,2,11,2,6,24.400000,7.573502,-1.281081,36.814423,-1.266467,36.792161,0


In [40]:
for df in [Train_df, Test_df]:
    df.drop(['Arrival at Pickup - Weekday (Mo = 1)', 'Confirmation - Weekday (Mo = 1)'], axis = 1, inplace = True)
    df.drop(['Arrival at Pickup - Day of Month', 'Confirmation - Day of Month'], axis = 1, inplace = True)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, shuffle=True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16960, 17) (4241, 17) (16960,) (4241,)


In [42]:
def test_model(model, y_test, X_test):
    predictions = model.predict(X_test.reshape(-1, 1))
    predictions = predictions.reshape(-1)
    rmsle = np.sqrt(np.mean((np.log(1+predictions) - np.log(1+y_test))**2))
    return np.round(rmsle, 3)

In [43]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [44]:
rs = 3
kfold = KFold(n_splits=10, random_state=rs, shuffle=True)

regressors = []
regressors.append(SVR())
regressors.append(GradientBoostingRegressor(random_state=rs))
regressors.append(ExtraTreesRegressor(n_estimators=rs))
regressors.append(RandomForestRegressor(random_state=rs))

cv_results = []
for regressor in regressors:     #scores to be minimised are negated (neg)
    cv_results.append(np.sqrt(abs(cross_val_score(regressor, X_train, y=y_train, scoring='neg_mean_squared_error', cv=kfold))))

cv_means = []
cv_stds = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_stds.append(cv_result.std())
    
cv_res = pd.DataFrame({ 
    "Algorithm": ["SVR", "GBR", "EXR", "RFR"],
    "CrossValMeans": cv_means, "CrossValErrors": cv_stds
                       })
cv_res = cv_res.sort_values("CrossValMeans", ascending=True)
print(cv_res)

  Algorithm  CrossValMeans  CrossValErrors
1       GBR     781.344527       14.927407
3       RFR     803.526323       13.391408
0       SVR     934.182015       17.032906
2       EXR     947.417393       14.933516


In [45]:
params = {
    'n_estimators':[75], # [75, 95],
    'num_leaves': [15], #[12,15, 17],
    'reg_alpha': [0.02], #[0.02, 0.05],
    'min_data_in_leaf': [300],  #[250, 280, 300]
    'learning_rate': [0.1], #[0.05, 0.1, 0.25],
    'objective': ['regression'] #['regression', None]
    }

lsearch = GridSearchCV(estimator = lgbm.LGBMRegressor(random_state=rs), cv=kfold,scoring='neg_mean_squared_error', param_grid=params)
lgbm = lsearch.fit(X_train, y_train)

l_params = lgbm.best_params_
l_score = np.sqrt(abs(lgbm.best_score_))
print(lgbm.best_params_, np.sqrt(abs(lgbm.best_score_)))

{'learning_rate': 0.1, 'min_data_in_leaf': 300, 'n_estimators': 75, 'num_leaves': 15, 'objective': 'regression', 'reg_alpha': 0.02} 776.310543663503


In [46]:
RFC = RandomForestRegressor(random_state=rs)
rf_param = {"max_depth":[None], "max_features":[3], "min_samples_split":[10],
           "min_samples_leaf": [3], "n_estimators":[300]}
rsearch = GridSearchCV(RFC, cv=kfold, scoring='neg_mean_squared_error',param_grid=rf_param)
rfm = rsearch.fit(X_train, y_train)

r_score = np.sqrt(abs(rfm.best_score_))
r_params = rfm.best_params_
print(r_score, r_params)

784.346530525453 {'max_depth': None, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}


In [47]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

lparams = {
           'learning_rate': 0.1, 'min_data_in_leaf': 300, 
           'n_estimators': 75, 'num_leaves': 20, 'random_state':rs,
           'objective': 'regression', 'reg_alpha': 0.02,
          'feature_fraction': 0.9, 'bagging_fraction':0.9}


lgbm = lgb.train(lparams, lgb_train, valid_sets=lgb_eval, num_boost_round=20, early_stopping_rounds=20)

lpred = lgbm.predict(X_test, num_iteration=lgbm.best_iteration)

print("The RMSE of prediction is ", mean_squared_error(y_test, lpred)**0.5)

[1]	valid_0's l2: 905931
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's l2: 852481
[3]	valid_0's l2: 809339
[4]	valid_0's l2: 773485
[5]	valid_0's l2: 744517
[6]	valid_0's l2: 720284
[7]	valid_0's l2: 700844
[8]	valid_0's l2: 685022
[9]	valid_0's l2: 672301
[10]	valid_0's l2: 661575
[11]	valid_0's l2: 652552
[12]	valid_0's l2: 644390
[13]	valid_0's l2: 639515
[14]	valid_0's l2: 633909
[15]	valid_0's l2: 629554
[16]	valid_0's l2: 625403
[17]	valid_0's l2: 622260
[18]	valid_0's l2: 619610
[19]	valid_0's l2: 616797
[20]	valid_0's l2: 614526
[21]	valid_0's l2: 612632
[22]	valid_0's l2: 611169
[23]	valid_0's l2: 609560
[24]	valid_0's l2: 607941
[25]	valid_0's l2: 606529
[26]	valid_0's l2: 605255
[27]	valid_0's l2: 604044
[28]	valid_0's l2: 603300
[29]	valid_0's l2: 602462
[30]	valid_0's l2: 601917
[31]	valid_0's l2: 600906
[32]	valid_0's l2: 600564
[33]	valid_0's l2: 600051
[34]	valid_0's l2: 599665
[35]	valid_0's l2: 599280
[36]	valid_0's l2: 598510
[37]	valid_

In [61]:
lgbm_y = lgbm.predict(test, num_iteration=lgbm.best_iteration)
lgbm_output = pd.DataFrame({"Order No":Test_df['Order No'], 
                           "Time from Pickup to Arrival": lgbm_y.astype(int) })
lgbm_output.to_csv("‪BlankSamplezindi.csv", index=False)
lgbm_output.head()

,Order No,Time from Pickup to Arrival
0,Order_No_19248,1562
1,Order_No_12736,1223
2,Order_No_768,1456
3,Order_No_15332,1247
4,Order_No_21373,1296


In [113]:
list_1 = Test_df['Order No'].tolist()
list_2 = lgbm_y
data = {"Order No": list_1, "Time from Pickup to Arrival": list_2}

df =pd.DataFrame(data)
df.head()

,Order No,Time from Pickup to Arrival
0,Order_No_19248,1562.825515
1,Order_No_12736,1223.538580
2,Order_No_768,1456.695780
3,Order_No_15332,1247.310613
4,Order_No_21373,1296.465783


In [116]:
df.to_csv(r"‪‪C:\Users\tulsi\Documents\BlankSamplezindi.csv", index =False, header = True)